In [1]:
import pandas as pd
import numpy as np

In [2]:
dataset=pd.read_csv("Social_Network_Ads.csv")

In [3]:
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [4]:
dataset=pd.get_dummies(dataset,dtype=int,drop_first=True)
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,1
1,15810944,35,20000,0,1
2,15668575,26,43000,0,0
3,15603246,27,57000,0,0
4,15804002,19,76000,0,1
...,...,...,...,...,...
395,15691863,46,41000,1,0
396,15706071,51,23000,1,1
397,15654296,50,20000,1,0
398,15755018,36,33000,0,1


In [5]:
dataset["Purchased"].value_counts()

Purchased
0    257
1    143
Name: count, dtype: int64

In [6]:
Independent=dataset[['User ID','Gender_Male','Age','EstimatedSalary']]
Independent

,User ID,Gender_Male,Age,EstimatedSalary
0,15624510,1,19,19000
1,15810944,1,35,20000
2,15668575,0,26,43000
3,15603246,0,27,57000
4,15804002,1,19,76000
...,...,...,...,...
395,15691863,0,46,41000
396,15706071,1,51,23000
397,15654296,0,50,20000
398,15755018,1,36,33000


In [7]:
Dependent=dataset[['Purchased']]
Dependent

,Purchased
0,0
1,0
2,0
3,0
4,0
...,...
395,1
396,1
397,1
398,0


In [8]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(Independent,Dependent,test_size=0.30,random_state=0)

In [9]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV

# Define the parameter grid
param_grid = {'alpha': [1, 2, 5],'fit_prior': [True, False]}

# Create the GridSearchCV object
grid = GridSearchCV(
    MultinomialNB(),
    param_grid,
    refit=True,
    verbose=3,
    n_jobs=-1,
    scoring='f1_weighted'
)

# Fit the model to the data
grid.fit(x_train, y_train)

# Print best parameters and best score
print("Best parameters found:", grid.best_params_)
print("Best score:", grid.best_score_)

Fitting 5 folds for each of 6 candidates, totalling 30 fits
Best parameters found: {'alpha': 1, 'fit_prior': True}
Best score: 0.6647321726994633


C:\Users\nandh\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [10]:
y_pred=grid.predict(x_test) #(.predict->defined function)
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test,y_pred)
print(cm)

[[49 30]
 [14 27]]


In [11]:
from sklearn.metrics import classification_report
clf_report=classification_report(y_test,y_pred)
print(clf_report)

              precision    recall  f1-score   support

           0       0.78      0.62      0.69        79
           1       0.47      0.66      0.55        41

    accuracy                           0.63       120
   macro avg       0.63      0.64      0.62       120
weighted avg       0.67      0.63      0.64       120



In [12]:
re=grid.cv_results_
grid_predictions = grid.predict(x_test) 
from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,y_pred,average='weighted')
print("f1_macro value for {}:".format(grid.best_params_),f1_macro)
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,grid.predict_proba(x_test)[:,1])
table=pd.DataFrame.from_dict(re)
table

f1_macro value for {'alpha': 1, 'fit_prior': True}: 0.6426080291271438


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_alpha,param_fit_prior,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.018295,0.004083,0.039552,0.006897,1,True,"{'alpha': 1, 'fit_prior': True}",0.607143,0.733523,0.638285,0.630424,0.714286,0.664732,0.049761,1
1,0.014663,0.006070,0.033454,0.000990,1,False,"{'alpha': 1, 'fit_prior': False}",0.607143,0.733523,0.638285,0.630424,0.714286,0.664732,0.049761,1
2,0.016811,0.008396,0.015491,0.004967,2,True,"{'alpha': 2, 'fit_prior': True}",0.607143,0.733523,0.638285,0.630424,0.714286,0.664732,0.049761,1
3,0.011708,0.002333,0.016127,0.003633,2,False,"{'alpha': 2, 'fit_prior': False}",0.607143,0.733523,0.638285,0.630424,0.714286,0.664732,0.049761,1
4,0.008033,0.003100,0.015190,0.004046,5,True,"{'alpha': 5, 'fit_prior': True}",0.607143,0.733523,0.638285,0.630424,0.714286,0.664732,0.049761,1
5,0.006766,0.005051,0.014909,0.003000,5,False,"{'alpha': 5, 'fit_prior': False}",0.607143,0.733523,0.638285,0.630424,0.714286,0.664732,0.049761,1
